### Exercise 7
Entrenamiento y ajuste fino de un árbol de decisión con el dataset moon.

- Se genera dataset moon con 10000 muestras y 0.4 de ruido

In [1]:
from sklearn.datasets import make_moons

In [2]:
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

- Se divide el dataset en train y test

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

- Se realiza búsqueda de rejilla (grid search) con validación cruzada para encontrar buenos hiperparámetros para un clasificador de árbol de decisión

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [20]:
### Hiperparámetros
# max_depth -> Profundidad del árbol
# min_samples_split -> Mínimo número de instancias que debe de tener el nodo
#                      para dividirlo
# min_samples_leaf -> Mínimo número de instancias en cada hoja
# min_weight_fraction_leaf -> Mismo objetivo que el anterior pero en %
# max_features -> Número máximo de características a buscar en cada nodo
# max_leaf_nodes -> Número máximo de nodos hoja

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)

grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [21]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=17, random_state=42)

In [22]:
grid_search_cv.best_params_

{'max_leaf_nodes': 17, 'min_samples_split': 2}

- Se entrena el clasificador con el conjunto de entrenamiento completo usando los hiperparámetros obtenidos del grid_search y se mide el rendimiento del modelo en el conjunto de test

In [18]:
# Por defecto, el grid search entrena el mejor modelo con el conjunto de entrenamiento completo y con la
# mejor combinación de hiperparámetros encontrados (desactivar con refit=False).

# rendimiento con el conjunto de entrenamiento completo
grid_search_cv.best_score_

0.8555001986342105

In [27]:
from sklearn.metrics import accuracy_score

In [20]:
y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

### Exercise 8
Clasificador Random Forest

Hacer crecer un bosque (forest). Continuación del ejercicio 7

- Se generan 1000 muestras del conjunto de entrenamiento donde cada una contiene 100 instancias seleccionadas aleatoriamente


In [6]:
from sklearn.model_selection import ShuffleSplit

In [15]:
n_samples = 1000 # número de árboles
n_instances = 100 # número de ejemplos de cada árbol
mini_sets = [(X_train[train_idx], y_train[train_idx]) for train_idx, test_idx in 
             ShuffleSplit(n_splits=n_samples, test_size=len(X_train) - n_instances, random_state=42).split(X_train)]

- Se entrena un árbol de decisión en cada muestra con los valores de los hiperparámetros encontrados en la búsqueda de rejilla y se evalúan los 1000 árboles en el conjunto de test.

In [23]:
from sklearn.base import clone

In [24]:
## Se crean 1000 objetos de árbol de decisión con los mejores hiperparámetros encontrados
forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_samples)]

In [25]:
forest[:5]

[DecisionTreeClassifier(max_leaf_nodes=17, random_state=42),
 DecisionTreeClassifier(max_leaf_nodes=17, random_state=42),
 DecisionTreeClassifier(max_leaf_nodes=17, random_state=42),
 DecisionTreeClassifier(max_leaf_nodes=17, random_state=42),
 DecisionTreeClassifier(max_leaf_nodes=17, random_state=42)]

In [29]:
import numpy as np

In [30]:
accuracy_scores = []
for tree, mini_train in zip(forest, mini_sets):
  tree.fit(mini_train[0], mini_train[1])

  y_pred = tree. predict(X_test)
  accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.8054499999999999

In [31]:
accuracy_scores[:5]

[0.7985, 0.836, 0.8065, 0.842, 0.7935]

- Para cada instancia del conjunto de test, se generan las predicciones de los 1000 árboles de decisión y se mantiene solo la predicción mas frecuente (voto mayoritario)

In [33]:
Y_pred = np.empty([n_samples, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [35]:
Y_pred

array([[0, 1, 0, ..., 0, 0, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 0]], dtype=uint8)

In [36]:
from scipy.stats import mode

In [37]:
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [38]:
y_pred_majority_votes

array([[1, 1, 0, ..., 0, 0, 0]], dtype=uint8)

- Se evalúan las predicciones en el conjunto de test

In [42]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.872